# __Find threshold to determine the MF threshold__

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from importlib import reload
import sys

from MF_plateauTest import MovingAv, rms_peak_ratio, fftspectrum
# reload(sys.modules['MF_plateauTest'])

# read all threshold for Yes/No values
All_TH = pd.read_csv('../DATA/All_Thresholds.csv', sep='\t')

# sampling rate 10kHz
sr = 10000

# take every N-th row (for skip_rows)
n=2
n_sr=sr/n

# subject to analyze
ID = 29
p_ID = "P"+str(ID)
table_threshold = pd.DataFrame(data=All_TH.query(f'ID=="P{ID}"')).reset_index(drop=True)

f_string = f'../DATA/Complete/Res_Cadence_P{ID}'

In [ ]:
plt.rcParams['figure.figsize'] = 10, 5
plt.rcParams['figure.max_open_warning']=50

sns.set_style("whitegrid")

## Load Labview file
read and downsample to 5kHz

In [ ]:
file_name_Lab = f_string+'.txt'

# count the lines
header_end = 23
num_lines = sum(1 for i, l in enumerate(open(file_name_Lab)) if i > header_end)

# the row indices to skip - make sure 0 is not included to keep header
skip_idx = list(range(header_end-1))
skip_idx.extend([x for x in range (header_end, num_lines) if (x-header_end) % n != 0])
lab_df = pd.read_csv(file_name_Lab,
                     skiprows=skip_idx, 
                     delimiter='\t',
                     usecols=['Untitled', 'Untitled 1', 'Untitled 2', 'Current'])
lab_df.columns=['Time', 'MF', 'Dial', 'Current']
lab_df.head()

## __Find and extract iterations__
 - find each iterations
 - smooth last second (24-25s) of probe data with moving average over 10 points
 - compute $\frac{rms}{peak}$
 - list all iterations < 0.65 and when phosphene have been perceived

In [ ]:
# find all iterations
stLab = lab_df.query('Time == 0')
idxLab = stLab.index.tolist()
idxLab.append(lab_df.index[-1])

# span = [18]
span = range(50)

for i in span:
    it_no = int(table_threshold.iloc[i,1])
    fr = table_threshold.query(f'Iteration=={it_no}')['Frequency'].item()
    
    section_Lab = np.arange(idxLab[:-1][i],idxLab[1:][i])
    
    data = lab_df.loc[section_Lab,:]
    data['Time'] = np.linspace(start=1/n_sr, stop=len(data)*1/n_sr, num=len(data))
    data['Iter_no'] = f'Iter{i+1}'

#     n_MF = n_MF.append(data[['Time', 'MF','Iter_no']])
    
    # check signal frequency
    if True:
        freq, sp = fftspectrum(data.MF, n_sr)
        mode_fr = freq[np.argmax(np.abs(sp.real))]
        
    
    len_nosignal = len(data.query('(Time>20) & (Dial<0.05)')) #  used Dial to check signal turned back to 0
    if len_nosignal > n_sr:
        Perceived = 0
    else:
        Perceived = int(table_threshold.loc[i,'Yes/No'])
        
    # compute rms Threshold
    if Perceived:
        subset = data.query('Time > 24')
        rmsf, _, rms_peak = rms_peak_ratio(MovingAv(subset['MF'],10))
        rms, _, _ = rms_peak_ratio(subset['MF']) if rms_peak > 0.65 else (None, None, None)
    else:
        rms = None
        rms_peak = None
            
    table_threshold.loc[i,'Perceived'] = int(Perceived)
    table_threshold.loc[i,'n_Threshold'] = rms
    table_threshold.loc[i,'rms/peak'] = rms_peak
    
    # graph
    if True:
        clrs = 'blue' if Perceived else 'red'
        data.plot(x='Time', y='MF', color=clrs)
        plt.title(f'P{ID} - iter# {it_no} - FR {fr}Hz - Perceived {Perceived} - THR {str(rms_peak):.4} - NoSignal: {len_nosignal}')
        plt.text(25.5,0, f'{mode_fr:.1f}Hz', {'fontweight':'bold', 'fontsize':'15'},color='k')

In [ ]:
table_threshold.style\
    .applymap(lambda x: 'background: darkorange' if x < 0.65 else " ", subset='rms/peak')\
    .bar(subset=['n_Threshold'], color='lightblue')

## __Test $\frac{rms}{peak}$ over sinus signal__
create sinus with ramp and noise to test the treshold of rms/peak ratio to detect a stable signal

In [ ]:
A = 80
duration=5
sr = 5000
t = np.linspace(0,duration,sr*duration)
fr = 50

coef_ramp=0.2
ramp = np.linspace(A,A+A*coef_ramp,len(t))

coef_noise=0.1
noise = (A*coef_noise*np.random.random(len(t)))


test_sin = ramp*np.sin(fr*t)+noise
test_sin = MovingAv(test_sin, 100)

rms = np.sqrt(np.mean(test_sin**2))
peak = np.max(test_sin)
r = rms/peak

plt.figure()
plt.plot(t, test_sin)
plt.title(f'ramp: {coef_ramp*100}% --- noise: {coef_noise*100}% --- ratio: {r:.3}')

r